In [25]:
from docplex.cp.model import *

def CP(tasks, edges, delays):
    # Instanciramo model
    mdl = CpoModel("MPCSD")

    num_tasks = len(tasks)
    tasks.sort()
    delays = dict(sorted(delays.items()))
    
    task_vars = [mdl.interval_var(name=f"{tasks[i]}",size=(1)) for i in range(num_tasks)] #Definisemo taskove kao intervale (u ovom slucaju su svi velicine 1, ali lako mozemo da prosledimo duzine ako zatreba)
    
    for edge in edges: #Zadajemo precedence constraint-ove kao i delay za svaki task
        mdl.add(mdl.end_before_start(task_vars[tasks.index(edge[0])],task_vars[tasks.index(edge[1])],delays[edge[0]]))
    
    sequence_vars = mdl.sequence_var([task_vars[i] for i in range(num_tasks)] , types = [i for i in range(num_tasks)], name = "sequence")
    mdl.add(mdl.no_overlap(sequence_vars)) #Bez preklapanja intervala
    objective=mdl.max([mdl.end_of(task_vars[i]) for i in range(num_tasks)])
    mdl.minimize(objective)
    # Solve the problem
    solution = mdl.solve()
    
    if solution:
        #solution.print_solution()
        return solution
    else:
        print("No solution found!")

In [28]:
tasks = ['A', 'B', 'C', 'D', 'E']
edges = [('A', 'B'), ('A', 'C'), ('B', 'D'), ('C', 'D'), ('D', 'E')]
delays = {'A': 1, 'B': 1, 'C': 4, 'D': 1, 'E': 3}
CP(tasks, edges, delays)

 ! --------------------------------------------------- CP Optimizer 22.1.1.0 --
 ! Minimization problem - 6 variables, 6 constraints
 ! Initial process time : 0.00s (0.00s extraction + 0.00s propagation)
 !  . Log search space  : 11.6 (before), 11.6 (after)
 !  . Memory usage      : 408.4 kB (before), 408.4 kB (after)
 ! Using parallel search with 6 workers.
 ! ----------------------------------------------------------------------------
 !          Best Branches  Non-fixed    W       Branch decision
                        0          6                 -
 + New bound is 10
 *            10        2  0.02s        1      (gap is 0.00%)
 ! ----------------------------------------------------------------------------
 ! Search completed, 1 solution found.
 ! Best objective         : 10 (optimal - effective tol. is 0)
 ! Best bound             : 10
 ! ----------------------------------------------------------------------------
 ! Number of branches     : 20
 ! Number of fails        : 65
 ! T

In [9]:
tasks = [
    "E", "Q", "G", "N", "F", "K", "M", "H", "I", "L", "S", "O", "B", "C", "P", "D", "J", "R", "A", "T"
]

edges = [
    ("E", "Q"), ("E", "G"), ("E", "N"), ("E", "A"), ("E", "C"), ("E", "R"), ("E", "S"), ("E", "D"), ("E", "B"), ("E", "O"), ("E", "K"), ("E", "P"), ("E", "I"), ("E", "T"), ("E", "J"), ("E", "F"), ("E", "H"),
    ("Q", "R"), ("Q", "J"), ("Q", "D"), ("Q", "T"), ("G", "N"), ("G", "T"), ("N", "R"), ("N", "L"), ("N", "D"), ("N", "K"), ("N", "M"), ("N", "P"), ("N", "F"), ("N", "S"), ("N", "O"), ("N", "H"), ("N", "A"),
    ("N", "C"), ("N", "B"), ("N", "J"), ("N", "I"), ("F", "D"), ("F", "T"), ("F", "C"), ("F", "A"), ("F", "S"), ("F", "J"), ("F", "H"), ("F", "K"), ("F", "O"), ("F", "I"), ("F", "P"), ("F", "M"), ("F", "L"),
    ("K", "T"), ("K", "H"), ("K", "L"), ("K", "A"), ("M", "I"), ("M", "L"), ("M", "P"), ("M", "B"), ("M", "H"), ("M", "D"), ("M", "C"), ("M", "O"), ("M", "T"), ("M", "A"), ("M", "S"), ("M", "R"), ("H", "C"),
    ("H", "B"), ("H", "S"), ("I", "P"), ("I", "A"), ("I", "T"), ("I", "S"), ("I", "J"), ("I", "O"), ("I", "L"), ("I", "C"), ("L", "J"), ("L", "A"), ("L", "D"), ("L", "C"), ("L", "S"), ("S", "R"), ("S", "T"),
    ("S", "A"), ("O", "C"), ("O", "P"), ("O", "A"), ("O", "B"), ("O", "J"), ("O", "R"), ("O", "T"), ("O", "D"), ("B", "A"), ("B", "R"), ("B", "T"), ("B", "P"), ("B", "D"), ("B", "C"), ("B", "J"), ("C", "D"),
    ("P", "T"), ("D", "J"), ("D", "R"), ("J", "R"), ("J", "T"), ("J", "A"), ("R", "A"), ("A", "T")
]

delays = {
    "E": 1, "Q": 5, "G": 5, "N": 0, "F": 0, "K": 2, "M": 2, "H": 5, "I": 3, "L": 1, "S": 5, "O": 4, "B": 4,
    "C": 2, "P": 2, "D": 5, "J": 2, "R": 5, "A": 2, "T": 5
}
CP(tasks, edges, delays)

 ! --------------------------------------------------- CP Optimizer 22.1.1.0 --
 ! Minimization problem - 21 variables, 111 constraints
 ! Initial process time : 0.00s (0.00s extraction + 0.00s propagation)
 !  . Log search space  : 86.4 (before), 86.4 (after)
 !  . Memory usage      : 447.5 kB (before), 447.5 kB (after)
 ! Using parallel search with 6 workers.
 ! ----------------------------------------------------------------------------
 !          Best Branches  Non-fixed    W       Branch decision
                        0         21                 -
 + New bound is 49
 ! Using iterative diving.
 *            49        4  0.02s        1      (gap is 0.00%)
 ! ----------------------------------------------------------------------------
 ! Search completed, 1 solution found.
 ! Best objective         : 49 (optimal - effective tol. is 0)
 ! Best bound             : 49
 ! ----------------------------------------------------------------------------
 ! Number of branches     : 18783
 !

In [26]:
def test_alg(path_to_test, path_to_results, alg, big_data=False): 
    files = os.listdir(path_to_test)
    data_files = [f for f in files if f.startswith('test_file_') and f.endswith('.json')]
    
    data_files.sort()
    
    data_to_write = []

    for file_name in data_files:
        file_path = os.path.join(path_to_test, file_name)
        with open(file_path, 'r') as f:
            loaded_data = json.load(f)
            tasks = loaded_data.get('tasks')
            edges = loaded_data.get('edges')
            delays = loaded_data.get('delays')
            
            solution = CP(tasks, edges, delays)
            
            draw_graph(tasks, edges, delays, file_name[:-5] + "_graph", big_data)
            print("----------------------------------------------")
            
            data = {'solution': solution}
            data_to_write.append(data)
    
    save_results(path_to_results, data_to_write)

In [27]:
def draw_graph(tasks, edges, delays, file_name, big_graphs=False, topological_order=None, save_path='graphs/'):
    G = nx.DiGraph()
    
    G.add_nodes_from(tasks)
    G.add_edges_from(edges)
    
    # Dynamic figure size based on the number of nodes
    num_nodes = len(tasks)
    if big_graphs:
        plt.figure(figsize=(12 + num_nodes * 0.2, 12 + num_nodes * 0.2))  # Adjust size dynamically
    else:
        plt.figure(figsize=(8, 8))  # Default size for small graphs
    
    if topological_order:
        # Ensure all nodes are in the topological order
        if len(topological_order) != len(tasks):
            raise ValueError("Topological order does not include all tasks.")
        
        # Use graphviz_layout for hierarchical drawing
        pos = nx.nx_agraph.graphviz_layout(G, prog='dot')
    else:
        pos = nx.spring_layout(G, k=0.5 if big_graphs else 0.8, iterations=50)
    
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    
    nx.draw(G, pos, with_labels=True, node_color='lightgreen', node_size=1000, font_size=16, font_color='black', arrowstyle='-|>', arrowsize=10)
    
    # Clipping the delays directly below the nodes
    for node, (x, y) in pos.items():
        plt.annotate(f"Delay: {delays[node]}",
                     xy=(x, y), xytext=(0, -20),  # Position text 20 points below the node
                     textcoords='offset points', ha='center', va='center',
                     bbox=dict(facecolor='white', alpha=0.5), fontsize=10, color='blue')

    file_path = os.path.join(save_path, file_name)
    plt.savefig(file_path)
    plt.show()